### Collab Notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip3 install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from simpletransformers.model import TransformerModel
PATH = 'drive/My Drive/Seq_Classification/'

In [ ]:
df = pd.read_csv(PATH+'task2_typ.csv')
df = df.rename(columns={'id':'Sentence #'})
df = df.dropna(axis=0)
df = df.sample(frac=1).reset_index(drop=True)
le = LabelEncoder()
labels = le.fit_transform(df['type'].values)
df['labels'] = labels
df.rename({'sentence':'text'},axis=1,inplace=True)
df = df[['text','labels','types']]

In [ ]:
n = len(df)
print(n)

In [ ]:
split = 0.8 
train = df[0:int(n*0.8)]
test = df[int(n*0.8):]
n_tags = len(set(df['labels'].values))
print("Train Size: {} \nTest Size: {} ".format(len(train),len(test)))

In [ ]:
def full_results(y_test,pred):
    
    report = classification_report(y_test,pred,output_dict=True)
    df = pd.DataFrame(report).transpose()
    display(df)
    
    inc_test = []
    exc_test = []
    inc_pred = []
    exc_pred = []
    for i,t in enumerate(typ_test):
        if t == 'I':
            inc_test.append(y_test[i])
            inc_pred.append(pred[i])
        else:
            exc_test.append(y_test[i])
            exc_pred.append(pred[i])
    print("For inclusions")
    report = classification_report(inc_test,inc_pred,output_dict=True)
    df = pd.DataFrame(report).transpose()
    display(df)
    print("For exclusions")
    report = classification_report(exc_test,exc_pred,output_dict=True)
    df = pd.DataFrame(report).transpose()
    display(df)
    return

args = {
   'output_dir': 'output/',
   'cache_dir': 'cache/',
   'fp16': False,
   'fp16_opt_level': 'O1',
   'max_seq_length': 128,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 5,
   'weight_decay': 0,
   'learning_rate': 3e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,'overwrite_output_dir': True,
   'reprocess_input_data': False,
#     'num_labels' : 4
}

## Train the model
train_data = []

for i,row in train.iterrows():
    train_data.append(list([row['text'],row['labels']]))

train_df = pd.DataFrame(train_data)

In [ ]:
model_args = ClassificationArgs(num_train_epochs=1)

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=n_tags,
    args=args
) 

model = TransformerModel('bert','bert-base-cased',args=args,use_cuda=True,num_labels=n_tags)
# Train the model
model.train_model(train_df)

In [ ]:
model.save_model(output_dir=PATH+'bert_task2',model=model.model)

In [ ]:
le = LabelEncoder()
labels = le.fit_transform(test['type2'].values)
test['labels'] = labels
model = ClassificationModel('bert',PATH+'bert_task2')
test_data = test['sentence'].values
pred = model.predict(test_data)[0]
labels = test['labels'].values
typ_test = test['type'].values
full_results(labels,pred)


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(labels,pred)

In [ ]:
le.inverse_transform(model.predict(['travel with the person you love'])[0])